### Tool Calling(Function Calling) 예제

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain
import string
# import os
# from dotenv import load_dotenv

# # API KEY 정보로드
# load_dotenv()


# os.environ["GOOGLE_CSE_ID"] = os.environ.get("GOOGLE_CSE_ID")
# os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0.1, 
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)


# 참고
# https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

@tool
def play_video(first_input: str) -> str:
    """영상 및 미디어 재생 관련 함수"""
    return "영상 재생 함수 실행"
    # words = [word.strip(string.punctuation) for word in first_input.split()]
    # video_input_list = ["틀어줘", "재생해줘", "영화", "드라마", "미디어"]
    # for word in words:
    #     if word in video_input_list:
    #         return "영상 재생 함수 실행"


@tool
def web_search(first_input: str) -> str:
    """웹검색 및 구체적인 정보 요청 관련 함수"""
    return "웹검색 함수 실행"
    # words = [word.strip(string.punctuation) for word in first_input.split()]
    # web_search_input_list = ["법률", "법규", "법제처", "검색", "검색해줘", "알려줘"]
    # for word in words:
    #     if word in web_search_input_list:
    #         return "웹검색 함수 실행"


# play_video 함수 호출 테스트
# play_video.invoke({
#     "first_input": "영화 타이타닉 틀어줘."
# })

# # web_search 함수 호출 테스트
# web_search.invoke({
#     "first_input": "소방법 관련 법규 알려줘."
# })


@tool
def test_chain_func(first_input: str) -> str:
    """요리 방법에 대한 답변 함수"""
    
    chat = ChatOpenAI(
        temperature=0.1,
        streaming=True,
        callbacks=[
            StreamingStdOutCallbackHandler(),
        ],
    )

    chef_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "",
            ),
            ("human", "I want to cook {cuisine} food."),
        ]
    )

    chef_chain = chef_prompt | chat


    veg_chef_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
            ),
            ("human", "{recipe}"),
        ]
    )


    veg_chain = veg_chef_prompt | chat


    final_chain = {"recipe": chef_chain} | veg_chain

    result = final_chain.invoke({"cuisine": first_input})

    return result.content


@tool
def google_search(first_input: str) -> str:
    """2023년 이후의 최신 정보 검색 관련 함수"""
    vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai")

    memory = ConversationSummaryBufferMemory(llm=llm, input_key='question', output_key='answer', return_messages=True, max_token_limit=30)

    search = GoogleSearchAPIWrapper()

    web_research_retriever = WebResearchRetriever.from_llm(
        vectorstore=vectorstore,
        llm=llm,
        search=search,
    )

    qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm, retriever=web_research_retriever)

    result = qa_chain({"question": first_input})

    return result["answer"]


In [2]:



# llm_with_tools = llm.bind_tools([play_video, web_search])
# msg = llm_with_tools.invoke("법제처")
# msg.tool_calls




# Tool Calling Agent
# tools = [play_video, web_search]
tools = [play_video, test_chain_func]

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 질문에 대한 의도를 정확하게 파악할 수 있는 전문가입니다. 질문이 오면 해당 질문의 맥락을 파악하여 그 의도를 대표할 수 있는 한 개의 단어로 대답해주세요."),
#     ("human", "{question}"),
#     MessagesPlaceholder(variable_name="agent_scratchpad")
# ])

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 대한 의도를 정확하게 파악할 수 있는 전문가입니다. 그리고 그 의도와 관련된 단어를 파생하는 능력도 출중합니다."),
    ("human", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# agent_executor.invoke({"question": "다비드 비야의 국가대표 통산 골 수 알려줘."})
# agent_executor.invoke({"question": "영화 타이타닉 틀어줘."})

# agent_executor.invoke({"question": "현재 OpenAI GPT의 가장 최신 모델 정보가 어떻게 돼?"})

agent_executor.invoke({"question": "일본의 요리 레시피가 궁금해."})

# agent_executor.invoke({"question": "리오넬 메시의 국가대표 통산 골 수 알려줘."})
# agent_executor.invoke({"question": "리오넬 메시의 국가대표 통산 골 수 알려줘. 정보는 2023년 이후의 최신 정보로 알려줘."})


# agent_executor.invoke({"question": "한국에서 스프링클러를 설치할 때 참고해야 하는 최신 법규가 궁금해."})

# agent_executor.invoke({"question": "한국에서 스프링클러를 설치할 때 참고해야 하는 법규가 궁금해. 참고해야하는 관련 법의 카테고리를 분류하고 각 카테고리에 대한 법률 정보를 한국의 법제처 사이트(https://www.moleg.go.kr/)를 참고해서 알려줘. 정보는 최신 정보였으면 좋겠어."})


# # Custom Output Parser
# def parse(ai_message: AIMessage) -> str:
#     """Parse the AI message."""
#     return ai_message.content



# chat = ChatOpenAI(
#     model="gpt-3.5-turbo-0125", 
#     temperature=0.1,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )



> Entering new AgentExecutor chain...

Invoking: `test_chain_func` with `{'first_input': '일본의 요리 레시피'}`


Sure! Here is a simple recipe for a popular Japanese dish called "Teriyaki Chicken":

Ingredients:
- 2 boneless, skinless chicken breasts
- 1/4 cup soy sauce
- 1/4 cup mirin (Japanese sweet rice wine)
- 2 tablespoons sugar
- 1 tablespoon vegetable oil
- 1 teaspoon grated ginger
- 1 clove garlic, minced
- Sesame seeds and chopped green onions for garnish

Instructions:
1. Slice the chicken breasts into thin strips or bite-sized pieces.
2. In a bowl, mix together the soy sauce, mirin, sugar, ginger, and garlic to make the teriyaki sauce.
3. Heat the vegetable oil in a large skillet over medium heat.
4. Add the chicken to the skillet and cook until browned on all sides.
5. Pour the teriyaki sauce over the chicken and stir to coat.
6. Cook for another 5-7 minutes, or until the chicken is cooked through and the sauce has thickened.
7. Serve the teriyaki chicken over steamed rice and g

{'question': '일본의 요리 레시피가 궁금해.',
 'output': '일본의 요리 레시피 중 채식주의자를 위한 테리야키 치킨 레시피를 알려드릴게요.\n\n재료:\n- 단단한 두부 블록 또는 세이탄, 얇게 썰거나 한 입 크기로 자른 것 1개\n- 간장 1/4컵\n- 미린 (일본식 달콤한 쌀 주류) 1/4컵\n- 설탕 2큰 숟가락\n- 식용유 1큰 숟가락\n- 다진 생강 1작은 숟가락\n- 다진 마늘 1쪽\n- 참깨와 다진 파초 (장식용)\n\n조리법:\n1. 만일 두부를 사용한다면, 물기를 제거하기 위해 압착하세요. 세이탄을 사용한다면 이 단계를 건너뛸 수 있습니다.\n2. 그릇에 간장, 미린, 설탕, 생강, 마늘을 섞어 테리야키 소스를 만듭니다.\n3. 대형 프라이팬에 식용유를 데워줍니다.\n4. 프라이팬에 두부나 세이탄을 넣고 갈색이 돌 때까지 익힙니다.\n5. 테리야키 소스를 두부나 세이탄 위에 부어 섞어줍니다.\n6. 또 5-7분 정도 더 익히면서 소스가 농도를 내고 두부나 세이탄이 따뜻해지도록 합니다.\n7. 증기를 푼 밥 위에 채식주의자용 테리야키 "치킨"을 담고 참깨와 다진 파초로 장식해주세요.\n\n이 채식주의자용 버전은 전통 요리의 맛을 그대로 즐기면서 고기가 없는 요리입니다. 집에서 만든 채식주의자용 테리야키 "치킨"을 즐겨보세요!'}